In [11]:
from logicGateNetwork import Layer, LogicGateNetwork
from geneticEngine import GeneticEngine
import numpy as np
from sklearn.metrics import classification_report

In [2]:
def evaluate_config(config: str, x: np.ndarray, y: np.ndarray, network: LogicGateNetwork) -> float:
    network.set_params(config)
    y_pred = network.forward(x)
    return sum(np.logical_xor(y, y_pred))

In [3]:
data = np.random.randint(2, size=(500, 16))

target_model = LogicGateNetwork([
    Layer(n_features=data.shape[1], units=16),
    Layer(n_features=16, units=8),
    Layer(n_features=8, units=1)
])

target_pred = target_model.forward(data)

training_model = LogicGateNetwork([
    Layer(n_features=data.shape[1], units=16),
    Layer(n_features=16, units=8),
    Layer(n_features=8, units=1)
])

conf_len = training_model.get_feature_vector_length()

In [4]:
conf_len

550

In [5]:
engine = GeneticEngine(
    evaluate_config,
    n_units=256,
    n_features=1,
    feature_size=conf_len,
    n_cut_points=int(conf_len / 44),
    n_mutate_points=int(conf_len / 44),
    cost_function_args=(data, target_pred, training_model)
)

In [6]:
engine.run(n_generations=250, verbose=True)

Current best fitness: 57.000000: 100%|██████████| 250/250 [00:58<00:00,  4.27it/s]     


In [7]:
print(evaluate_config(engine.get_best()[0], data, target_pred, training_model))

[57]


In [8]:
training_model.set_params(engine.get_best()[0])

In [9]:
preds = training_model.forward(data)

In [13]:
print(classification_report(y_true=target_pred, y_pred=preds))

              precision    recall  f1-score   support

       False       0.75      0.44      0.56        81
        True       0.90      0.97      0.93       419

    accuracy                           0.89       500
   macro avg       0.83      0.71      0.75       500
weighted avg       0.88      0.89      0.87       500

